In [0]:
#import libraries
import math
import re
from collections import defaultdict, Counter
from nltk import ngrams
import pandas as pd
import glob as gb
import numpy as np
import matplotlib.pyplot as plt

1a starts here

In [0]:
#this method does the tokenization and removal of numbers and punctuations
def tokenize(text):
    "List all the word tokens (consecutive letters) in a text. Normalize to lowercase."
    return re.findall('[a-z]+', text.lower())

In [0]:
#this method handles reading the training corpus
def readCorpus(directory="ag_news_csv_cleaned/train_cleaned.csv"):
  with open(directory, 'r',encoding="utf8", errors='replace') as doc:
      data = doc.readlines()
  return data

In [0]:
#read corpus method is invoked here
training_data=readCorpus()

In [0]:
#divide sentence to their categories 1,2,3,4 they are saved respectively to list in index 0,1,2,3 which is a list of lists
catSentList=[[],[],[],[]]
for x in training_data:
  catSentList[int(x[1:2])-1].append(tokenize(x))

In [0]:
#merge category list gotten above to a universal list for each category
import more_itertools
reducedList=[]
for i in range(len(catSentList)):
  reducedList.append(list(more_itertools.flatten(catSentList[i])))

In [0]:
#count the words in each group/category
countDictList=[]
for i in range(len(reducedList)):
  countDictList.append(Counter(reducedList[i]))

In [0]:
#this takes the training data as a whole instead of filtering and grouping them 
#universal count
listofSentList=[]
for x in training_data:
  listofSentList.append(tokenize(x))
#convert the list of list into just one which is a universal list
flatSentList=list(more_itertools.flatten(listofSentList))
#count word occurence in the list and remove words that have counts less than 3
sentCount=Counter(flatSentList)
vocabulary=Counter(dict(filter(lambda x: x[1] >= 2, sentCount.items())))

1a ends here and 1b starts here

In [0]:
# b From the preprocessed text, collect the following statistics
D=len(listofSentList) #the total number of documents
Dk= [len(catSentList[0]),len(catSentList[1]),len(catSentList[2]),len(catSentList[3]) ] #the total number of documents labelled with class k+1
#the frequency of word wt in the documents of class k
dictVocab = dict(vocabulary)
vocabularyCat = []
#copy the vabulary and replicate it for each categories
for i in range(len(Dk)):
  vocabularyCat.append(dictVocab.copy())

In [0]:
#compute the N_K for each category, that is the frequency of word w t in the documents of class k
for i in range(len(vocabularyCat)):
  for k,v in vocabularyCat[i].items(): 
    vocabularyCat[i][k] = countDictList[i][k]

In [0]:
# c Estimate the priors P(C_k) 
priorsPCK = [] 
catLength = [] 
for i in range(len(Dk)):
  priorsPCK.append(Dk[i]/D)
  catLength.append(sum(vocabularyCat[i].values()))
  

In [0]:
# d Estimate the likelihoods P(w_t|C_k) as
def Likelihood(word,CK=1,alpha=1):
  likelihd = (alpha + countDictList[CK-1][word])/(len(vocabulary) + catLength[CK-1])
  return likelihd
  

In [0]:
# Design and implement a Python structure NB_Classifier that encapsulates the model
# parameters mentioned above, namely the priors and the likelihoods.
def NB_Classifier2(wordseq):
  classCat = []
  for i in range(len(Dk)):
    firstPart = math.log(priorsPCK[i],2)
    #the second part is a sum over 
    secondpart = sum([math.log(Likelihood(j,CK = i+1),2) for j in wordseq])
    classCat.append(firstPart+secondpart)
  return classCat.index(max(classCat))+1
      

In [0]:
#this method handles reading the training corpus
def readTestCorpus(directory="ag_news_csv_cleaned/test_cleaned.csv"):
  with open(directory, 'r',encoding="utf8", errors='replace') as doc:
      data = doc.readlines()
  return data

In [0]:
#read test corpus method is invoked here
test_data=readTestCorpus()

In [0]:
len(test_data)

7600

In [0]:
# e Apply the same prepossessing steps that you have used on the training portion on the test
#portion. Using your method in part (e), predict the class for every test document.
classlabel = []
predlabel = []
testSet = []
for x in test_data:
  classlabel.append(int(x[1:2]))
  testSet.append(tokenize(x))
  
  

In [0]:
#generate the predicted classes per document
for document in testSet:
  predlabel.append(NB_Classifier2(document))

In [0]:
#calculate the accuracy, get the number of times the lists matches 
correctPred=sum(a == b for a,b in zip(classlabel, predlabel))
print("correct predictions = ",correctPred)
accuracy =  correctPred/len(classlabel)
print("The accuracy is = ", accuracy)


correct predictions =  6822
The accuracy is =  0.8976315789473684


In [0]:
#confusion matrix
import pandas as pd
y_actu = pd.Series(classlabel, name='Actual')
y_pred = pd.Series(predlabel, name='Predicted')
df_confusion = pd.crosstab(y_actu, y_pred)

In [0]:
df_confusion

Predicted,1,2,3,4
Actual,,,,
1,1697,63,92,48
2,25,1854,7,14
3,70,19,1599,212
4,70,18,140,1672


3 and 4 get confused more

 Design and implement a Python structure NB_Classifier that encapsulates the model
parameters mentioned above, namely the priors and the likelihoods.
def NB_Classifier(wordseq):
  classCat = []
  for i in range(len(Dk)):
    firstPart = math.log(priorsPCK[i],2)
    #the second part is a sum over 
    secondpart=0
    for j in wordseq:
      secondpart+=math.log(Likelihood(j,CK = i+1),2)
    classCat.append(firstPart+secondpart)
  return classCat.index(max(classCat))+1